In [ ]:
TYPE = 'track' # artist or track
TIME_RANGE = 'long_term' # short_term, medium_term, long_term
LIMIT = 50 # 1-50; the code can handle more than 50, but the Spotify API, at least currently, only allows 50

In [ ]:
import spotipy

import requests
import math
import os
import json
from image_color import get_html_colors

with open('token.txt', 'r') as f:
    token = f.read()

sp = spotipy.Spotify(auth=token)

In [ ]:
top_items = None

# get top tracks or artists
if TYPE == 'artist':
    top_items = sp.current_user_top_artists(limit=LIMIT, time_range=TIME_RANGE)
elif TYPE == 'track':
    top_items = sp.current_user_top_tracks(limit=LIMIT, time_range=TIME_RANGE)

if LIMIT > 50:
    print('\033[33m' + '[WARNING] Spotify API only allows 50 items right now.' + '\033[0m')
    offset = 50
    while offset < LIMIT:
        api_limit = 50
        if offset + 50 > LIMIT:
            api_limit = LIMIT - offset

        if TYPE == 'artist':
            top_items['items'] += sp.current_user_top_artists(limit=api_limit, time_range=TIME_RANGE, offset=offset)['items']
        elif TYPE == 'track':
            top_items['items'] += sp.current_user_top_tracks(limit=api_limit, time_range=TIME_RANGE, offset=offset)['items']
        offset += 50

print('\033[32m' + 'Got top items' + '\033[0m')

In [ ]:
from pprint import pprint
pprint(top_items)

In [ ]:
# extract image url and id from each item and save it to a list
images = []

if top_items is None or len(top_items['items']) == 0:
    print('\033[91m' + '[Error] No items found' + '\033[0m')
    exit()

if len(top_items['items']) < LIMIT:
    len_items = len(top_items['items'])
    print('\033[93m' + f'[Warning] Not enough items found [Only {len_items} found]' + '\033[0m')

if TYPE == 'artist':
    for item in top_items['items']:
        images.append({
            'id': item['id'],
            'name': item['name'],
            'url': item['images'][0]['url']
        })
elif TYPE == 'track':
    for item in top_items['items']:
        artists = ''
        for artist in item['artists']:
            artists += artist['name'] + ', '
        artists = artists[:-2]
        images.append({
            'id': item['id'],
            'name': '"' + item['name'] + '"\nby\n' + artists,
            'url': item['album']['images'][0]['url']
        })

print('\033[92m' + '[Success] Found {} images'.format(len(images)) + '\033[0m')

In [ ]:
# download images
if not os.path.exists('images'):
    os.mkdir('images')

if not os.path.exists(f'images/{TYPE}s'):
    os.mkdir(f'images/{TYPE}s')

# create json file with mapping between id and name
if os.path.exists(f'images/{TYPE}s.json'):
    with open(f'images/{TYPE}s.json', 'r') as f:
        json_data = json.load(f)
else:
    json_data = {}

for image in images:
    json_data[image['id']] = image['name']

with open(f'images/{TYPE}s.json', 'w') as f:
    json.dump(json_data, f)

download_count = 0
for image in images:
    if os.path.exists(f'images/{TYPE}s/{image["id"]}.jpg'):
        continue
    r = requests.get(image['url'])
    download_count += 1
    with open(f'images/{TYPE}s/{image["id"]}.jpg', 'wb') as f:
        f.write(r.content)
existing_count = len(images) - download_count

print(f'\033[92m' + f'[Success] Downloaded {download_count} new images [{existing_count} were already downloaded]' + '\033[0m')

In [ ]:
# generate html file

template_top, template_bottom = None, None
with open('templates/top.html', 'r') as f:
    template_top = f.read()
with open('templates/bottom.html', 'r') as f:
    template_bottom = f.read()

if not os.path.exists(f'generated'):
    os.mkdir(f'generated')

with open(f'generated/{TYPE}s_{TIME_RANGE}_size{LIMIT}.html', 'w', encoding='utf-8') as f:

    f.write(template_top)

    TYPE_TITLE = TYPE.title()

    mapped_TIME_RANGEs = {
        'short_term': 'Last 4 Weeks',
        'medium_term': 'Last 6 Months',
        'long_term': 'All Time'
    }

    TIME_RANGE_TITLE = mapped_TIME_RANGEs[TIME_RANGE]
    f.write(f'<h1 style="text-align: center; color: white; font-family: Consolas;">Your Top {TYPE_TITLE}s [{TIME_RANGE_TITLE}]</h1>')

    f.write('<table>')
    length_of_line = int(math.sqrt(len(images)))

    html_colors = get_html_colors([image['id'] for image in images], f'images/{TYPE}s/')

    for x in range(0, length_of_line):
        f.write('<tr>')
        for y in range(0, length_of_line):
            name = images[x*length_of_line+y]['name'].replace('\n', '<br>')
            track_id = images[x*length_of_line+y]['id']
            text_color = html_colors[track_id]['text_color']
            background_color = html_colors[track_id]['background_color']
            f.write('<td>')
            f.write(f'<a href="https://open.spotify.com/{TYPE}/{track_id}">')
            f.write(f'<img src="../images/{TYPE}s/{track_id}.jpg" width="100" height="100">')
            f.write(f'<span style="color: {text_color}; background-color: {background_color};">{name}</span>')
            f.write('</a>')
            f.write('</td>')
        f.write('</tr>')
    f.write('</table>')

    f.write(template_bottom)
    if TYPE == 'artist':
        f.write(f'<h1 style="text-align: center; color: white; font-family: Consolas;">List of Your Artists</h1>')
    elif TYPE == 'track':
        f.write(f'<h1 style="text-align: center; color: white; font-family: Consolas;">List of Your Tracks</h1>')
    
    f.write('<table style="border-collapse: collapse;">')
    if TYPE == 'artist':
        for i in range(len(top_items['items'])):
            item = top_items['items'][i]
            f.write('<tr style="border-bottom: 1pt dashed gray; height: 100px">')
            f.write(f'<td class="footer" style="text-align: right; padding-right: 25px;">{i+1}.</td>')
            f.write(f'<td style="padding-right: 25px"><img src="../images/{TYPE}s/{item["id"]}.jpg" width="50" height="50"></td>')
            f.write(f'<td style="padding-right: 15px"><a class="footer" href="https://open.spotify.com/artist/{item["id"]}">{item["name"]}</a></td>')
            popularity = item['popularity']
            color = f'rgb({255 - int(popularity * 2.55)}, {int(popularity * 2.55)}, 0)'
            f.write(f'<td class="footer poptooltip">[<text style="color: {color};">&#9210; {popularity}</text>]</td>')
            f.write('</tr>')
    elif TYPE == 'track':
        for i in range(len(top_items['items'])):
            item = top_items['items'][i]
            f.write('<tr style="border-bottom: 1pt dashed gray; height: 80px">')
            f.write(f'<td class="footer" style="text-align: right; padding-right: 25px;">{i+1}.</td>')
            f.write(f'<td style="padding-right: 25px"><img src="../images/{TYPE}s/{item["id"]}.jpg" width="50" height="50"></td>')
            artists = ''
            for artist in item['artists']:
                artists += artist['name'] + ', '
            artists = artists[:-2]
            f.write(f'<td style="padding-right: 15px; max-width: 600px; overflow: hidden; text-overflow: ellipsis; white-space: wrap;"><a class="footer" href="https://open.spotify.com/track/{item["id"]}">{item["name"]}</a><br><text class="footer" style="font-size: 16px;">{artists}</text></td>')
            popularity = item['popularity']
            color = f'rgb({255 - int(popularity * 2.55)}, {int(popularity * 2.55)}, 0)'
            f.write(f'<td class="footer poptooltip">[<text style="color: {color};">&#9210; {popularity}</text>]</td>')
            f.write('</tr>')

    f.write('</table>')
    f.write('</body>')
    f.write('</html>')

print('\033[92m' + f'[Success] Done! Created file: {TYPE}s_{TIME_RANGE}_size{LIMIT}.html' + '\033[0m')
project_dir = os.getcwd().replace('\\', '/')
print('\033[92m' + f'[Success] URL to file: file:///{project_dir}/generated/{TYPE}s_{TIME_RANGE}_size{LIMIT}.html' + '\033[0m')